# Import libs

In [ ]:
import os
import glob
import pandas as pd
from datetime import datetime
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## GP registered population data (Region)

In [ ]:
# data source: https://digital.nhs.uk/data-and-information/publications/statistical/patients-registered-at-a-gp-practice/june-2021
`
# set the path to data
data_path = r'data/'
data_file = r'gp_reg_pop_regions.csv'
gpreg_df = pd.read_csv(data_path+data_file)

# format region names
gpreg_df['region_name'] = gpreg_df['region_name'].str.replace(' COMMISSIONING REGION', '')
gpreg_df['region_name'] = gpreg_df['region_name'].str.upper()
gpreg_df

## NHS App

In [ ]:
import os
import glob
import pandas as pd

def concat_datasets(path):
    """import multiple datasets (.csv files) and concatenates into one dataframe"""
    # glob makes a list of all files and paths to each dataset i.e., ".../data/dataset1.csv" and so on
    file_list = glob.glob(os.path.join(data_path, "*.csv"))      
    # for each file (.csv) in the list of files concatenate them together    
    df = pd.concat((pd.read_csv(file) for file in file_list))
    # returns the single dataframe as the output of the function
    return df

# set the path to data folder
data_path = r'data/NHS_app_extract'
# invoke function and save results as nhsapp_df
nhsapp_df = concat_datasets(data_path)

# remove pre-processing prefixes
prefixes = ['First_','Max_','Sum_']
for prefix in prefixes:
    nhsapp_df.columns = nhsapp_df.columns.str.replace(prefix, '')
nhsapp_df['Region'] = nhsapp_df['Region'].str.upper()

# set cols to keep
cols = ['Month','Report_Date','Region','NHSApp_RegistrationsCount', 'Usage_LoginSessions_Login_Sessions',
                'Usage_Appointments_Appointments_booked',
                'Usage_CancelledAppointments_Cancellation_Count',
                'Usage_MedicalRecords_Medical_record_views',
                'Usage_Prescriptions_Prescriptions_Ordered']
                
# make df from selected cols
nhsapp_df = pd.DataFrame(nhsapp_df, columns=cols)

nhsapp_df.tail()

In [ ]:
nhsapp_df = nhsapp_df.groupby(['Report_Date','Region']).sum().reset_index()
nhsapp_df

In [ ]:
# join GP reg population data to NHS App dataset
nhsapp_df = pd.merge(nhsapp_df, gpreg_df, left_on='Region', right_on='region_name', how='left')

nhsapp_df

In [ ]:
# set cols that relate to activity metrics
activity_cols = ['Usage_LoginSessions_Login_Sessions',
                'Usage_Appointments_Appointments_booked',
                'Usage_CancelledAppointments_Cancellation_Count',
                'Usage_MedicalRecords_Medical_record_views',
                'Usage_Prescriptions_Prescriptions_Ordered']

# set col that contains the new user registration figures
registration_col = ['NHSApp_RegistrationsCount']
       
# set col that contains the GP population figures
population_col = 'NUMBER_OF_PATIENTS'

def per100kpopulation(df, cols, population, per_capita):
    """divide the calculation_cols by the population_col on a per capita basis"""
    per100kpop = [col + "_per100kpop" for col in cols]
    df[per100kpop] = df[cols].div(df[population]/per_capita, axis=0)
    return df

# set per capita value i.e., per 1000 or per 100K population
per_capita = 100000

# invoke function and save results as nhsapp_df
nhsapp_df = per100kpopulation(nhsapp_df, activity_cols+registration_col, population_col, per_capita)

In [ ]:
start_date = '2019-06-01'
end_date = '2021-06-01'
# convert the 'report date' col to datetime format
nhsapp_df['Report_Date'] = pd.to_datetime(
    nhsapp_df['Report_Date']
    )

# create a mask that will filter by date
mask = (
    nhsapp_df['Report_Date'] > start_date) & (
    nhsapp_df['Report_Date'] <= end_date
    )
# apply the date filter to the dataframe
nhsapp_df = nhsapp_df.loc[mask]

data_path = r'data/NHS_app_extract/outputs'
nhsapp_df.to_csv(os.path.join(data_path, 'nhsapp_df_analysis.csv'))

## POMI

In [ ]:
# import data
data_path = r'data/Pomi_latest'

# invoke function and save results as nhsapp_df
pomi_df = concat_datasets(data_path)

In [ ]:
pomi_df = pomi_df.groupby(['region_name','field', 'report_period_end'])['value'].sum().unstack('field').reset_index()

In [ ]:
# format region names
pomi_df['region_name'] = pomi_df['region_name'].str.replace(' COMMISSIONING REGION', '')
pomi_df['region_name'] = pomi_df['region_name'].str.upper()

# join GP reg population data to POMI dataset
pomi_df = pd.merge(pomi_df, gpreg_df, left_on='region_name', right_on='region_name', how='left')
pomi_df

In [ ]:
per_capita = 100000

pomi_df['Pat_Appts_per100kpopEnbld'] = pomi_df['Pat_Appts_Use'].div(pomi_df['Pat_Appts_Enbld']/per_capita, axis=0)

pomi_df['Pat_Presc_per100kpopEnbld'] = pomi_df['Pat_Appts_Use'].div(pomi_df['Pat_Presc_Enbld']/per_capita, axis=0)

pomi_df['Total_Use_per100kpopEnbld'] = pomi_df['Total_Use'].div(pomi_df['Total_Pat_Enbld']/per_capita, axis=0)

pomi_df

In [ ]:
start_date = '2019-06-01'
end_date = '2021-06-01'
# convert the 'report date' col to datetime format
pomi_df['report_period_end'] = pd.to_datetime(
    pomi_df['report_period_end']
    )

# create a mask that will filter by date
mask = (
    pomi_df['report_period_end'] > start_date) & (
    pomi_df['report_period_end'] <= end_date
    )
# apply the date filter to the dataframe
pomi_df = pomi_df.loc[mask]

data_path = r'data/Pomi_latest/outputs'
pomi_df.to_csv(os.path.join(data_path, 'pomi_df_analysis.csv'))

## GP Survey

In [ ]:
# import data
data_path = r'data/GP_survey'
file_name = r'GPPS_2021_CCG_data_(weighted)_(csv)_PUBLIC.csv'
file_list = glob.glob(os.path.join(data_path, file_name))
gpSurvey_df = pd.concat((pd.read_csv(file) for file in file_list))
gpSurvey_df.columns

In [ ]:
# import data
data_path = r'data/GP_survey'
file_name = r'GPPS_2021_List_of_reporting_variables_(csv)_PUBLIC.csv'
file_list = glob.glob(os.path.join(data_path, file_name))
gpSurvey_questions_df = pd.concat((pd.read_csv(file) for file in file_list))
gpSurvey_questions_df.head(10)

## Appointment Types

In [ ]:
import os
import glob
import pandas as pd

def concat_datasets(path):
    file_list = glob.glob(os.path.join(data_path, "*.csv"))      
    df = pd.concat((pd.read_csv(file) for file in file_list))
    return df

data_path = r'/Users/mary.amanuel/Desktop/digital health insights/appointments/CCG_CSV_Apr_21.csv'
appointments_df = pd.read_csv(data_path)
appointments_df.head(10)

In [ ]:
appointments_grouped = appointments_df.groupby(['Appointment_Date', 'APPT_MODE'])['COUNT_OF_APPOINTMENTS'].sum().unstack('APPT_MODE').reset_index()
appointments_grouped